# Web API

Run this notebook in your browser using [Binder](https://mybinder.org/v2/gh/flexcompute-readthedocs/tidy3d-docs/readthedocs?labpath=docs%2Fsource%2Fnotebooks%2FWebAPI.ipynb).

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
import tidy3d as td

## Setup

Let's set up a simple simulation to get started.

In [2]:
# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1/fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]),
    medium=dielectric)

# create source
source = td.VolumeSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time = td.GaussianPulse(
        freq0=freq0,
        fwidth=fwidth),
    polarization='Ex')

# create monitor
monitor = td.FieldMonitor(
    fields=['Ex', 'Ey', 'Ez'],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name='field')

# Initialize simulation
sim = td.Simulation(size=sim_size,
                    grid_size=(dl, dl, dl),
                    structures=[square],
                    sources=[source],
                    monitors=[monitor],
                    run_time=run_time,
                    pml_layers=(pml, pml, pml))

## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
import tidy3d.web as web

# upload the simulaition to our servers
task_id = web.upload(sim, task_name='webAPI')

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, path='data/sim.hdf5')

Using Tidy3D credentials from stored file


[13:08:56] INFO     Uploaded task 'webAPI' with task_id                         ]8;id=508303;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=638858;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#108\108]8;;\
                    '3b8095d1-c128-436d-85af-ddfb132c0a2d'.

Output()

[13:08:57] status = queued                                                      ]8;id=860850;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=703443;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#200\200]8;;\

[13:09:09] status = preprocess                                                  ]8;id=504026;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=550951;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#200\200]8;;\

...

[13:10:29] INFO     downloading file "monitor_data.hdf5" to "data/sim.hdf5"     ]8;id=70671;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=54389;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#479\479]8;;\

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

           INFO     loading SimulationData from data/sim.hdf5                   ]8;id=930084;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=5875;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#300\300]8;;\

           WARNING  Simulation final field decay value of 0.336 is greater than ]8;id=104304;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=431587;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#307\307]8;;\
                    the simulation shutoff threshold of 1e-05. Consider                      
                    simulation again with large run_time duration for more                   
                    accurate results.

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the web.run() function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We won't run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to `web.start()` and then exit the session and load the results at a later time using `web.load`.


## Job Container

The `web.Job` interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original `Simulation`.

While `Job` has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [4]:
# set the logging level to warning to reduce downloading messages.
td.set_logging_level('warning')

# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name='job')

# start job, monitor, and load results
sim_data = job.run(path='data/sim.hdf5')

Output()

[13:10:42] status = preprocess                                               ]8;id=306220;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=691517;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#142\142]8;;\

[13:11:00] status = running                                                  ]8;id=75320;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=61336;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#142\142]8;;\

[13:11:14] status = postprocess                                              ]8;id=515739;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=211520;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#142\142]8;;\

[13:11:31] status = visualize                                                ]8;id=953200;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=724461;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#142\142]8;;\

[13:12:01] status = success                                                  ]8;id=981273;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:]8;id=495655;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py#142\142]8;;\

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

[13:12:02] WARNING  Simulation final field decay value of 0.336 is greater than ]8;id=460895;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=12505;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#307\307]8;;\
                    the simulation shutoff threshold of 1e-05. Consider                      
                    simulation again with large run_time duration for more                   
                    accurate results.

Another convenient thing about `Job` objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [5]:
# saves the job metadata to a single file
job.to_file('data/job.json')

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file('data/job.json')

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path='data/sim.hdf5')

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

[13:12:04] WARNING  Simulation final field decay value of 0.336 is greater than ]8;id=873410;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=726672;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#307\307]8;;\
                    the simulation shutoff threshold of 1e-05. Consider                      
                    simulation again with large run_time duration for more                   
                    accurate results.

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in `Batch()` object.
The batch object is like a `Job()`, but stores task metadata for a series of simulations.

In [6]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f'sim_{i}': sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir='data')

Output()

sim_0: status = success ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
sim_1: status = success ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
sim_2: status = success ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

When the batch is completed, the output is not a dictionary of `SimulationData` but rather a python generator that can be looped through once to get the simulation data one by one.

This was chosen to reduce the memory strain from loading all `SimulationData` objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [7]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
batch.download(path_dir='data')
for task_name, sim_data in batch.items(path_dir='data'):
    field_data = sim_data.at_centers('field').sel(f=freq0)
    intensity = abs(field_data.Ex)**2 + abs(field_data.Ey)**2 + abs(field_data.Ez)**2
    sum_intensity = float(intensity.sum(('x', 'y')).values)
    intensities[task_name] = sum_intensity

print(intensities)

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Output()

...

[13:14:13] WARNING  Simulation final field decay value of 0.336 is greater than ]8;id=312105;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=94452;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#307\307]8;;\
                    the simulation shutoff threshold of 1e-05. Consider                      
                    simulation again with large run_time duration for more                   
                    accurate results.

Output()

Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

[13:14:14] WARNING  Simulation final field decay value of 0.336 is greater than ]8;id=147632;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:]8;id=176724;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py#307\307]8;;\
                    the simulation shutoff threshold of 1e-05. Consider                      
                    simulation again with large run_time duration for more                   
                    accurate results.

{'sim_0': 12626396.596452318, 'sim_1': 12626393.573843408, 'sim_2': 12626396.596452318}
